1. #### Matlab SPM pipeline for making brain regions and tumor regions in uncorrected, corrected and ground truth DSC space

In [1]:
# Run once to store stdout
import sys
nb_stdout = sys.stdout

In [2]:
# Redirect stdout to console, to not get too much text output in the notebook
# This means that the notebook will not output any text. Text will be redirected
# to the terminal where the notebook was started.
sys.stdout = open(1, 'w')

In [3]:
output_directory_suffix = "2019_07_02_native"
corrections_base_directory = "../epi_corrections_out_" + output_directory_suffix
corrections_base_directory

'../epi_corrections_out_2019_07_02_native'

In [4]:
from os import getcwd
from os.path import abspath
epi_corrections_root_directory = abspath(getcwd() + "/..") # Go one directory up from current working directory
epi_corrections_root_directory

'/media/loek/HDD3TB1/data/IVS_EPI_BASELINE/epi_corrections'

In [11]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory" "$output_directory_suffix"
epi_corrections_root_directory=$1
corrections_base_directory=../$2
output_directory_suffix=$3

spm_path=/media/loek/HDD3TB1/apps/spm12 # This needs to be changed
cd $epi_corrections_root_directory
pipeline_report_file=$2/pipeline_report_spm_$output_directory_suffix.txt
run_command='matlab 
                -nodisplay 
                -nosplash 
                -nodesktop 
                -r "cd('"'spm_pipeline'"'); 
                    addpath('"'$spm_path'"'); 
                    myCluster = parcluster; 
                    myCluster.NumWorkers = 4; 
                    saveProfile(myCluster); 
                    parpool(4); 
                    make_dsc_rois('"'$corrections_base_directory'"'); 
                    exit;" 
                2>&1 | tee $pipeline_report_file'
eval $run_command

To see the log, run from corrections_base_directory

tail -f pipeline_report_spm*.txt

2. #### Copy over EPI_raw_DSC, EPI_applytopup and EPI_applyepic to a separate directory

In [12]:
corrections_base_new_directory = \
"../epi_corrections_out_2019_07_02_native_tumor_excluded_from_rcbv"

3. #### Run remove-tumor-from-cbv.ipynb on this separate directory to remove rCBV based on tumor GT-ROIs

4. #### compute-roi-medians.sh using brain and tumor GT ROIs

In [36]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_new_directory"
epi_corrections_root_directory=$1
corrections_base_new_directory=$2
cd $epi_corrections_root_directory
command="bash scripts/compute-roi-medians.sh \
$corrections_base_new_directory \
2>&1 | tee $corrections_base_new_directory/computegtroismedianslog.txt"
eval $command

5. #### compute-dice-between-rois.sh using brain and tumor GT and uncorrected and corrected ROIs

In [40]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_new_directory"
epi_corrections_root_directory=$1
corrections_base_new_directory=$2
cd $epi_corrections_root_directory
command="bash scripts/compute-dice-between-rois.sh \
$corrections_base_new_directory \
2>&1 | tee $corrections_base_new_directory/computedicelog.txt"
eval $command

6. #### analyze-medians-and-dice-scores.sh

7. #### analyze-dice-gt-cor.sh